In [1]:
import pandas as pd
import sys
import dateutil
from mpd import MPDClient
from tqdm import tqdm

sys.path.append('../..')
pd.set_option('display.max_columns', 500)

In [2]:
mpd = MPDClient()
mpd.connect("localhost", 6600)

In [3]:
data = mpd.listallinfo()
data = [datum for datum in data if 'directory' not in datum]
df = pd.DataFrame(data)

In [4]:
def get_year(datum):
    if datum['originaldate']:
        return dateutil.parser.parse(datum['originaldate']).year
    if datum['date']:
        return dateutil.parser.parse(datum['date']).year
    return None

df['year'] = df.apply(get_year, axis=1)
df.duration = df.time
df['album_artist'] = df.albumartist
df.head(4)

,file,last-modified,format,time,duration,artist,albumartist,artistsort,albumartistsort,title,album,track,date,originaldate,genre,disc,label,musicbrainz_albumid,musicbrainz_artistid,musicbrainz_albumartistid,musicbrainz_releasetrackid,musicbrainz_trackid,composer,performer,year,album_artist
0,After Forever/2005 - Remagine_ The Album - The...,2020-07-26T17:26:21Z,44100:24:2,66,66,After Forever,After Forever,After Forever,After Forever,Enter,Remagine: The Album - The Sessions,1,2015,2005,Metal,1,Transmission Records,2e373838-8a3b-402a-95ad-c32a3653ff3d,6b966946-5274-4e3d-aabb-7563814d805f,6b966946-5274-4e3d-aabb-7563814d805f,60057e8a-82b7-410d-9bc1-6af91690fce9,e8905475-fe30-42c7-97a5-cc9471bc7fb7,NaN,NaN,2005,After Forever
1,After Forever/2005 - Remagine_ The Album - The...,2020-07-26T17:26:22Z,44100:24:2,302,302,After Forever,After Forever,After Forever,After Forever,Come,Remagine: The Album - The Sessions,2,2015,2005,Metal,1,Transmission Records,2e373838-8a3b-402a-95ad-c32a3653ff3d,6b966946-5274-4e3d-aabb-7563814d805f,6b966946-5274-4e3d-aabb-7563814d805f,b22dae0d-1c12-4088-afb0-c49df9e61d1a,55b875cf-4175-46e2-a24e-7fea00085b06,NaN,NaN,2005,After Forever
2,After Forever/2005 - Remagine_ The Album - The...,2020-07-26T17:26:22Z,44100:24:2,225,225,After Forever,After Forever,After Forever,After Forever,Boundaries Are Open,Remagine: The Album - The Sessions,3,2015,2005,Metal,1,Transmission Records,2e373838-8a3b-402a-95ad-c32a3653ff3d,6b966946-5274-4e3d-aabb-7563814d805f,6b966946-5274-4e3d-aabb-7563814d805f,fe1f0ac2-59fc-4832-9590-80386da66feb,0d4f8d1f-c663-4585-b779-faba67613ac0,NaN,NaN,2005,After Forever
3,After Forever/2005 - Remagine_ The Album - The...,2020-07-26T17:26:22Z,44100:24:2,253,253,After Forever,After Forever,After Forever,After Forever,Living Shields,Remagine: The Album - The Sessions,4,2015,2005,Metal,1,Transmission Records,2e373838-8a3b-402a-95ad-c32a3653ff3d,6b966946-5274-4e3d-aabb-7563814d805f,6b966946-5274-4e3d-aabb-7563814d805f,0e8ee49e-021d-47f1-8f7d-ea5c4498263e,adcd690e-87d6-4da0-8d71-c6e1900dbaa7,NaN,NaN,2005,After Forever


In [5]:
from api import DBConn
from models import Base, MpdSong

DBConn()

In [6]:
DBConn.engine.execute('CREATE SCHEMA IF NOT EXISTS mpd')
tables = []
for name, table in Base.metadata.tables.items():
    if table.schema == 'mpd':
        tables.append(table)
Base.metadata.create_all(DBConn.engine, tables)

In [7]:
with DBConn.get_session() as db:
    tracks = list(df.itertuples(index=False))
    for track in tqdm(tracks):
        track = track._asdict()
        song = MpdSong(**{k:v for k, v in track.items() if k in MpdSong.__table__.columns.keys()})
        
        added = db.query(MpdSong).filter_by(file=track['file']).first()
        if not added:
            db.merge(song)
    db.commit()

100%|██████████| 1544/1544 [10:32<00:00,  2.44it/s]
